In [1]:
import pandas as pd
import re #use regex expressions to separate characters

df_euro = pd.read_csv('https://query.data.world/s/tp4rtb4ryxelnxhbftjtoqf7khpcw7?dws=00000')
df_euro_positions = pd.read_csv("https://query.data.world/s/v5qjxh7wpbff5b7dz7ouo3x4b5aiyo?dws=00000")
df_matches = pd.read_csv("https://query.data.world/s/fhxv7rlrmcmxle4ktj77f54c7kq426?dws=00000")

# column to search 
column_name = 'StatsName'  
search_strings = [
        "Goals", "Total Attempts", "Attempts on target", "Attempts off target", "Attempts Accuracy",
        "Saves", "Tackles won", "Clearances", "Recovered balls", "Passes completed", "Passes accuracy",
        "Distance covered (m)", "Top Speed (Km/h)", "Own-goals", "Goals conceded", "Attempts blocked",
        "Attempts on bar", "Attempts on post", "Passes attempted", "Dribbling", "Tackles", "Tackles lost",
        "Blocks", "Played Time", "Fouls suffered", "Fouls committed", "Assists", "Big Chances"]

# Escape each string in the search_strings list
escaped_search_strings = [re.escape(string) for string in search_strings]

# Create a regex pattern that matches any of the escaped search strings
search_pattern = '|'.join(escaped_search_strings)

# filtering the df to only include rows where the column contains any of the search strings
df_euro_stats = df_euro[df_euro[column_name].str.contains(search_pattern, na=False, regex=True)]
df_euro_stats.reset_index(drop=True, inplace=True)


#display(df_euro_stats)



In [2]:
df_positions = df_euro_positions[['ID', 'Role']]   # so now i have the 2nd dataset and want to reduce it to only ID and Role
df_positi = df_positions[['ID', 'Role']].rename(columns={'ID': 'PlayerID'}) #renaming ID to PlayerID to match the other dataset
 
df_positi.reset_index(inplace=True)


df_europivot = df_euro_stats.pivot_table(index=['PlayerID', 'PlayerName', 'PlayerSurname', 'HomeTeamName', 'AwayTeamName','MatchID',"PlayedTime"],
                                         columns='StatsName',
                                         values='Value',
                                         aggfunc='sum')     #pivoting the 1st dataset to better undertsatnd the stats

#reset index
df_europivot.reset_index(inplace=True)

# Display the pivoted DataFrame

#display(df_positi)
#display(df_europivot)

In [3]:

# merging the datasets based on PlayerID
role_merge= pd.merge(df_europivot, df_positi[['PlayerID', 'Role']], on='PlayerID', how='left')

role_merge.rename(columns={'Role_y': 'Role'}, inplace=True) #merged_df got the Role as Role_y so i had to change it 


#merging datasets based on MatchID

matchid_merge = pd.merge(role_merge, df_matches[["MatchID" ,"RoundName"]], on = "MatchID", how= "left")
df_euro_all_stats= matchid_merge

In [4]:
# reorganize the columns
desired_columns_order = ['PlayerID', 'PlayerName', 'PlayerSurname', 'HomeTeamName', 'AwayTeamName','MatchID',"RoundName","PlayedTime", "Role", 'Goals', 'Assists', 'Saves',
                         'Total Attempts', 'Attempts on target', 'Attempts off target', 'Attempts Accuracy', "Attempts blocked","Attempts on bar", "Attempts on post",'Big Chances', 
                         'Dribbling', 'Tackles','Tackles won', 'Fouls suffered', 'Fouls committed', 'Blocks', 'Clearances', 'Recovered balls', 'Passes completed',
                         'Passes accuracy', 'Distance covered (m)', 'Top Speed (Km/h)', 'Own-goals', 'Goals conceded']

# reorder the columns in euro_stats_df
df_euro_all_stats = df_euro_all_stats[desired_columns_order]


In [27]:
#functions

def clean_time(df): 
    """ Clean the time column of the DataFrame to a valid format.
    args: df: The input DataFrame containing the 'PlayedTime' column to be cleaned.
    return: df: A copy of the input DataFrame with the 'PlayedTime' column cleaned and formatted to 'min'.
    """
    df1 = df.copy()
    df1['PlayedTime'] = df1['PlayedTime'].apply(lambda x: f"{int(x) *130 // 8142 }")
    #since the PlayedTime was given in a weird way i decided to equal the max palyed time to the 130 min (the max time played in a game of the tournment)
    df1['PlayedTime'] = pd.to_numeric(df1['PlayedTime'], errors='coerce')
    df1.insert(8,"Games-p90", df1["PlayedTime"] / 90) # creating a new column with games played per90min
               
    return df1


def calculate_statistics(df):
    """
    Calculate the combined statistics for each player across multiple games.
    Args:
        df (DataFrame): DataFrame containing player data with multiple games.
    Returns:
        DataFrame: DataFrame with combined statistics for each player.
    """
    # Define the columns for aggregation
    aggregation_columns = [
        "Games-p90", "Goals", "Assists", "Own-goals", "Goals conceded",
        "Total Attempts", "Attempts on target", "Attempts off target", "Saves", "Tackles won", 
        "Clearances", "Recovered balls", "Passes completed", "Distance covered (m)", 
        "Attempts blocked", "Attempts on bar", "Attempts on post", "Dribbling", "Tackles",
        "Blocks", "Fouls suffered", "Fouls committed", "Big Chances", "Attempts Accuracy",
        "Passes accuracy", "Top Speed (Km/h)"
    ]

    # Filter out non-existing columns
    aggregation_columns = [col for col in aggregation_columns if col in df.columns]

    # Convert columns to numeric where applicable
    for column in aggregation_columns:
        df[column] = pd.to_numeric(df[column], errors='coerce')

    # Group by PlayerID and calculate sum, mean, and max for respective columns
    df_summed = df.groupby('PlayerID')[aggregation_columns].agg({
        "Games-p90": "sum",
        "Goals": "sum",
        "Assists": "sum",
        "Own-goals": "sum",
        "Goals conceded": "sum",
        "Total Attempts": "sum",
        "Attempts on target": "sum",
        "Attempts off target": "sum",
        "Saves": "sum",
        "Tackles won": "sum",
        "Clearances": "sum",
        "Recovered balls": "sum",
        "Passes completed": "sum",
        "Distance covered (m)": "sum",
        "Attempts blocked": "sum",
        "Attempts on bar": "sum",
        "Attempts on post": "sum",
        "Dribbling": "sum",
        "Tackles": "sum",
        "Blocks": "sum",
        "Fouls suffered": "sum",
        "Fouls committed": "sum",
        "Big Chances": "sum",
        "Attempts Accuracy": "mean",
        "Passes accuracy": "mean",
        "Top Speed (Km/h)": "max"
    }).reset_index()

    # Calculate per 90 statistics
    for col in aggregation_columns:
        if col != "PlayerID" and col != "Games-p90" and col != "Top Speed (Km/h)" and col != "Attempts Accuracy" and col != "Passes accuracy":
            df_summed[col + ' per 90'] = (df_summed[col] / df_summed["Games-p90"]).round(2)

            
    #dropping the columns that are not needed
    df_summed.drop(columns=["Total Attempts", "Attempts on target", "Attempts off target", "Saves", "Tackles won", 
        "Clearances", "Recovered balls", "Passes completed", "Distance covered (m)", 
        "Attempts blocked", "Attempts on bar", "Attempts on post", 
        "Dribbling", "Tackles", "Blocks", "Fouls suffered", 
        "Fouls committed", "Big Chances"], inplace=True)
    
    return df_summed



def merge_player_info(df, df_summed):
    """
    Merge the player information with the calculated statistics.
    Args:
        df (DataFrame): Original DataFrame containing player data with multiple games.
        df_summed (DataFrame): DataFrame with combined statistics for each player.
    Returns:
        DataFrame: Final DataFrame with player information and combined statistics.
    """
    # Get player information
    df_player_info = df[["PlayerID", "PlayerName", "PlayerSurname", "Role"]].drop_duplicates()

    # Merge the summed stats with player information
    df_final = pd.merge(df_player_info, df_summed, on='PlayerID')

    return df_final

def sum_stats(df):
    """
    Combine the statistics for each player across multiple games and merge with player information.
    Args:
        df (DataFrame): DataFrame containing player data with multiple games.
    Returns:
        DataFrame: Final DataFrame with combined statistics and player information.
    """
    df_summed = calculate_statistics(df)
    df_final = merge_player_info(df, df_summed)
    display(df_final)
    return df_final

In [28]:
df_games=clean_time(df_euro_all_stats) #clean time
df_games.drop_duplicates(inplace=True)
df_games.reset_index(inplace=True)
display(df_games)

df_per90=sum_stats(df_games)  # dataset with all stats combined from all games
#print(df3)

,index,PlayerID,PlayerName,PlayerSurname,HomeTeamName,AwayTeamName,MatchID,RoundName,PlayedTime,Games-p90,...,Fouls committed,Blocks,Clearances,Recovered balls,Passes completed,Passes accuracy,Distance covered (m),Top Speed (Km/h),Own-goals,Goals conceded
0,0,52148,Goran,Pandev,Austria,North Macedonia,2024442,final tournament,92,1.022222,...,1,0,0,0,18,69,10073,29.5,0,0
1,3,52148,Goran,Pandev,North Macedonia,Netherlands,2024445,final tournament,67,0.744444,...,0,0,0,0,12,67,7257,26.5,0,0
2,6,52148,Goran,Pandev,Ukraine,North Macedonia,2024444,final tournament,91,1.011111,...,0,0,0,0,14,61,9824,27,0,0
3,9,59142,Giorgio,Chiellini,Belgium,Italy,2024486,quarter finals,95,1.055556,...,1,1,7,13,36,88,9628,29,0,0
4,14,59142,Giorgio,Chiellini,Italy,England,2024491,final,130,1.444444,...,3,0,5,10,111,95,12687,25.2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1392,6134,250147581,Kevin,Nisbet,England,Scotland,2024461,final tournament,6,0.066667,...,0,0,0,0,2,100,909,24.3,0,0
1393,6137,250147581,Kevin,Nisbet,Scotland,Czech Republic,2024452,final tournament,15,0.166667,...,1,0,0,0,1,33,1779,25,0,0
1394,6140,250155055,Ché,Adams,Croatia,Scotland,2024472,final tournament,82,0.911111,...,0,0,0,0,9,75,8674,28.1,0,0
1395,6143,250155055,Ché,Adams,England,Scotland,2024461,final tournament,81,0.900000,...,2,0,1,1,22,85,8441,31.3,0,0


,PlayerID,PlayerName,PlayerSurname,Role,Games-p90,Goals,Assists,Own-goals,Goals conceded,Attempts Accuracy,...,Distance covered (m) per 90,Attempts blocked per 90,Attempts on bar per 90,Attempts on post per 90,Dribbling per 90,Tackles per 90,Blocks per 90,Fouls suffered per 90,Fouls committed per 90,Big Chances per 90
0,52148,Goran,Pandev,forwards,2.777778,1,0.0,0,0,16.666667,...,9775.44,0.00,0.0,0.0,0.00,0.00,0.00,2.88,0.36,0.00
1,59142,Giorgio,Chiellini,defenders,5.055556,0,0.0,0,0,10.000000,...,9385.71,0.00,0.0,0.0,0.20,0.59,0.40,0.20,0.99,0.00
2,62487,Sebastian,Larsson,midfielders,4.133333,0,0.0,0,0,37.500000,...,10755.97,0.00,0.0,0.0,0.24,1.45,0.48,0.24,0.73,0.00
3,63706,Cristiano,Ronaldo,forwards,4.155556,5,1.0,0,0,50.000000,...,8698.48,0.72,0.0,0.0,2.41,0.24,0.00,2.17,0.00,0.24
4,64005,David,Marshall,goalkeepers,3.011111,0,0.0,0,5,0.000000,...,4737.79,0.00,0.0,0.0,0.00,0.33,0.00,0.33,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,250143242,Kalvin,Phillips,midfielders,7.533333,0,1.0,0,0,14.285714,...,11016.77,0.00,0.0,0.0,0.40,1.73,0.00,0.53,1.59,0.00
437,250143313,Tamás,Cseri,midfielders,0.200000,0,0.0,0,0,0.000000,...,11600.00,0.00,0.0,0.0,5.00,5.00,0.00,0.00,0.00,0.00
438,250143795,Martin,Koscelník,defenders,1.188889,0,0.0,0,0,0.000000,...,9515.61,0.00,0.0,0.0,4.21,1.68,0.00,1.68,1.68,0.00
439,250147581,Kevin,Nisbet,forwards,0.344444,0,0.0,0,0,16.666667,...,11412.58,2.90,0.0,0.0,0.00,0.00,0.00,0.00,2.90,0.00


In [ ]:
#df3.info()


In [52]:
def defenders_performance(df):
    df_copy = df.copy()

    def calculate_defenders_score(row):
        if row['Role'] == 'defenders':
            weights = {
                "Goals" : 1,
                "Assists" : 0.6,
                "Total Attempts" : 0.05,
                "Attempts on target" : 0.15,
                "Dribbling" : 0.05,
                'Blocks': 0.1,
                'Clearances': 0.1,
                'Own-goals': -0.5,
                "Fouls suffered" : 0.1,
                "Fouls committed" : -0.1,
                'Tackles': 0.1,
                "Distance covered (m)" : 0.0001
            }
            # adjust weights based on RoundName
            round_factors = {
                "final tournament": 1,
                "eighth finals": 1.3,
                "quarter finals": 1.6,
                "semi finals": 1.8,
                "final": 2,
            }
            if row["RoundName"] in round_factors:
                factor = round_factors[row["RoundName"]]
                adjusted_weights = {stat: weight * factor for stat, weight in weights.items()}
            else:
                adjusted_weights = weights
            
            # Calculate combined score using adjusted weights
            combined_score = sum(row[stat] * adjusted_weights[stat] for stat in weights)
            return combined_score
        else:
            return None

    df_copy['defenders_score'] = df_copy.apply(calculate_defenders_score, axis=1)
    mean_defenders_scores = df_copy.groupby(['PlayerName', 'PlayerSurname'], as_index=False)['defenders_score'].mean()
    top_10_defenders_by_mean_score = mean_defenders_scores.nlargest(10, 'defenders_score')
    
    for index, row in top_10_defenders_by_mean_score.iterrows():
        print(f"{row['PlayerName']} {row['PlayerSurname']}: {row['defenders_score']:.2f}")

# Make sure to replace df_games with the actual DataFrame variable you're using
defenders_performance(df_games) 

Harry Maguire: 3.74
Luke Shaw: 2.86
Joakim Mæhle: 2.84
Stefan Lainer: 2.60
Giovanni Di Lorenzo: 2.51
Leonardo Spinazzola: 2.38
Giorgio Chiellini: 2.36
Nico Elvedi: 2.33
Leonardo Bonucci: 2.32
César Azpilicueta: 2.29


In [53]:
def midfielders_performance(df):
    df_copy = df.copy()

    def calculate_midfielders_score(row):
        if row['Role'] == 'midfielders':
            weights = {
                "Goals" : 1.3,
                "Assists" : 0.5,
                "Total Attempts" : 0.05,
                "Attempts on target" : 0.15,
                "Dribbling" : 0.07,
                'Blocks': 0.1,
                'Clearances': 0.1,
                "Recovered balls" :0.1,
                "Passes completed" : 0.005,
                'Own-goals': -0.5,
                "Fouls suffered" : 0.1,
                "Fouls committed" : -0.1,
                'Tackles': 0.1,
                "Distance covered (m)" : 0.0001
            }
            # adjust weights based on RoundName
            round_factors = {
                "final tournament": 1,
                "eighth finals": 1.3,
                "quarter finals": 1.6,
                "semi finals": 1.8,
                "final": 2,
            }
            if row["RoundName"] in round_factors:
                factor = round_factors[row["RoundName"]]
                adjusted_weights = {stat: weight * factor for stat, weight in weights.items()}
            else:
                adjusted_weights = weights
            
            # Calculate combined score using adjusted weights
            combined_score = sum(row[stat] * adjusted_weights[stat] for stat in weights)
            return combined_score
        else:
            return None

    df_copy['midfielders_score'] = df_copy.apply(calculate_midfielders_score, axis=1)
    mean_midfielders_scores = df_copy.groupby(['PlayerName', 'PlayerSurname'], as_index=False)['midfielders_score'].mean()
    top_10_midfielders_by_mean_score = mean_midfielders_scores.nlargest(10, 'midfielders_score')
    
    for index, row in top_10_midfielders_by_mean_score.iterrows():
        print(f"{row['PlayerName']} {row['PlayerSurname']}: {row['midfielders_score']:.2f}")

# Make sure to replace df_games with the actual DataFrame variable you're using
midfielders_performance(df_games) 

Marco Verratti: 4.96
Mislav Oršic: 3.82
Emil Forsberg: 3.59
Pierre-Emile Højbjerg: 3.42
Thorgan Hazard: 3.40
Georginio Wijnaldum: 3.33
Kalvin Phillips: 3.24
Paul Pogba: 3.23
Dani Olmo: 3.14
Taras Stepanenko: 2.99


In [59]:
def forwards_performance(df):
    df_copy = df.copy()

    def calculate_forwards_score(row):
        if row['Role'] == 'forwards':
            weights = {
                "Goals" : 1.5,
                "Assists" : 0.5,
                "Total Attempts" : 0.05,
                "Attempts on target" : 0.15,
                "Dribbling" : 0.1,
                'Blocks': 0.1,
                'Clearances': 0.1,
                "Recovered balls" :0.1,
                "Passes completed" : 0.005,
                'Own-goals': -0.5,
                "Fouls suffered" : 0.1,
                "Fouls committed" : -0.1,
                'Tackles': 0.1,
                "Distance covered (m)" : 0.0001
            }
            # adjust weights based on RoundName
            round_factors = {
                "final tournament": 1,
                "eighth finals": 1.3,
                "quarter finals": 1.6,
                "semi finals": 1.8,
                "final": 2,
            }
            if row["RoundName"] in round_factors:
                factor = round_factors[row["RoundName"]]
                adjusted_weights = {stat: weight * factor for stat, weight in weights.items()}
            else:
                adjusted_weights = weights
            
            # Calculate combined score using adjusted weights
            combined_score = sum(row[stat] * adjusted_weights[stat] for stat in weights)
            return combined_score
        else:
            return None

    df_copy['forwards_score'] = df_copy.apply(calculate_forwards_score, axis=1)
    mean_forwards_scores = df_copy.groupby(['PlayerName', 'PlayerSurname'], as_index=False)['forwards_score'].mean()
    top_10_forwards_by_mean_score = mean_forwards_scores.nlargest(10, 'forwards_score')
    
    for index, row in top_10_forwards_by_mean_score.iterrows():
        print(f"{row['PlayerName']} {row['PlayerSurname']}: {row['forwards_score']:.2f}")

# Make sure to replace df_games with the actual DataFrame variable you're using
forwards_performance(df_games) 

Cristiano Ronaldo: 4.30
Harry Kane: 4.15
Raheem Sterling: 3.93
Patrik Schick: 3.67
Lorenzo Insigne: 3.59
Karim Benzema: 3.53
Mikkel Damsgaard: 3.45
Álvaro Morata: 3.28
Kasper Dolberg: 3.23
Jeremy Doku: 3.22


In [57]:
def goalkeepers_performance(df):
    df_copy = df.copy()

    def calculate_goalkeepers_score(row):
        if row['Role'] == 'goalkeepers':
            weights = {
                "Goals" : 1.0,
                "Assists" : 0.5,
                "Total Attempts" : 0.05,
                "Attempts on target" : 0.15,
                "Dribbling" : 0.1,
                'Blocks': 0.1,
                'Clearances': 0.1,
                "Recovered balls" :0.01,
                "Passes completed" : 0.005,
                'Own-goals': -0.5,
                "Fouls suffered" : 0.05,
                "Fouls committed" : -0.2,
                'Tackles': 0.1,
                "Distance covered (m)" : 0.0001,
                "Saves" : 0.3,
                "Goals conceded" : -0.5
            }
            # adjust weights based on RoundName
            round_factors = {
                "final tournament": 1,
                "eighth finals": 1.3,
                "quarter finals": 1.6,
                "semi finals": 1.8,
                "final": 2,
            }
            if row["RoundName"] in round_factors:
                factor = round_factors[row["RoundName"]]
                adjusted_weights = {stat: weight * factor for stat, weight in weights.items()}
            else:
                adjusted_weights = weights
            
            # Calculate combined score using adjusted weights
            combined_score = sum(row[stat] * adjusted_weights[stat] for stat in weights)
            return combined_score
        else:
            return None

    df_copy['goalkeepers_score'] = df_copy.apply(calculate_goalkeepers_score, axis=1)
    mean_goalkeepers_scores = df_copy.groupby(['PlayerName', 'PlayerSurname'], as_index=False)['goalkeepers_score'].mean()
    top_10_goalkeepers_by_mean_score = mean_goalkeepers_scores.nlargest(10, 'goalkeepers_score')
    
    for index, row in top_10_goalkeepers_by_mean_score.iterrows():
        print(f"{row['PlayerName']} {row['PlayerSurname']}: {row['goalkeepers_score']:.2f}")

# Make sure to replace df_games with the actual DataFrame variable you're using
goalkeepers_performance(df_games) 

Jordan Pickford: 2.14
Yann Sommer: 1.57
Thibaut Courtois: 1.45
Kasper Schmeichel: 1.32
Tomáš Vaclík: 1.27
Gianluigi Donnarumma: 1.21
Lukas Hradecky: 1.17
Robin Olsen: 1.11
David Marshall: 1.06
Daniel Bachmann: 0.94


In [32]:

#df_games["RoundName"].value_counts()

RoundName
final tournament    967
eighth finals       231
quarter finals      115
semi finals          53
final                31
Name: count, dtype: int64

In [33]:
#df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1397 entries, 0 to 1396
Data columns (total 36 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 1397 non-null   int64  
 1   PlayerID              1397 non-null   int64  
 2   PlayerName            1397 non-null   object 
 3   PlayerSurname         1397 non-null   object 
 4   HomeTeamName          1397 non-null   object 
 5   AwayTeamName          1397 non-null   object 
 6   MatchID               1397 non-null   int64  
 7   RoundName             1397 non-null   object 
 8   PlayedTime            1397 non-null   int64  
 9   Games-p90             1397 non-null   float64
 10  Role                  1397 non-null   object 
 11  Goals                 1397 non-null   int64  
 12  Assists               1389 non-null   float64
 13  Saves                 1389 non-null   float64
 14  Total Attempts        1389 non-null   float64
 15  Attempts on target   